In [20]:
import os
import torch
from torchvision import transforms
from PIL import Image
from model_architecture import CowClassifier

In [21]:
# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Crear instancia del modelo
model = CowClassifier().to(device)

# Cargar los pesos en el modelo
model.load_state_dict(torch.load('../../models/classifier/cow_class_model_state.pth', map_location=device))
model.eval()

C:\Users\dembo\AppData\Local\Temp\ipykernel_11564\2687114777.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../../models/classifier/co

CowClassifier(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=100352, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.5, inplace=False)
    (13): Linear(in_features=128, out_features=64, bias=True)
    (14): ReLU()
    (15): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [22]:
# Transform for test images (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Class labels
class_labels = {0: 'acostada', 1: 'parada'}

# Directory to process
test_directory = '../../dataset/classifier/detect/'  # Set to the directory containing images for prediction

# Counters for prediction results
total_images = 0
class_counts = {label: 0 for label in class_labels.keys()}

In [23]:
# Process each image in the directory
for filename in os.listdir(test_directory):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
        img_path = os.path.join(test_directory, filename)
        
        try:
            # Open and transform the image
            image = Image.open(img_path).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)
            
            # Run the model on the image
            output = model(image)
            _, pred_label = output.max(1)
            pred_label = pred_label.item()
            
            # Update counts and display result
            class_counts[pred_label] += 1
            total_images += 1
            print(f"Image: {filename} - Predicted class: {class_labels[pred_label]}")      
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

# Summary
print("\nPrediction Summary:")
print(f"Total images processed: {total_images}")
for label, count in class_counts.items():
    print(f"Number of images in '{class_labels[label]}' class: {count}")

Image: val_0.jpg - Predicted class: acostada
Image: val_1.jpg - Predicted class: parada
Image: val_2.jpg - Predicted class: acostada
Image: val_3.jpg - Predicted class: acostada
Image: val_4.jpg - Predicted class: acostada
Image: val_5.jpg - Predicted class: parada
Image: val_6.jpg - Predicted class: parada

Prediction Summary:
Total images processed: 7
Number of images in 'acostada' class: 4
Number of images in 'parada' class: 3
